In [1]:
import pandas
from psycopg2.extensions import QuotedString

In [2]:
homedir = %env HOME
%load_ext sql
%config SqlMagic.short_errors=False
%config SqlMagic.dsn_filename='{homedir}/odbc.ini'  # Read from DSN config file with commonly used connections.
%config SqlMagic.autopandas=True
%sql [local_mnist]

In [3]:
d = %sql -l   # Share same connection between SqlMagic and psycopg2
conn = list(d.values())[0].session.connection.connection     # Get psycopg2 Connection object from within SqlAlchemy within SqlMagic
conn.autocommit = True                                       # Automatically commit after every call to %sql or cur.execute()
cur = conn.cursor()

In [4]:
%sql -l

{'postgresql://dominovaldano@localhost:6000/mnist': <sql.connection.Connection at 0x10bbef730>}

## Define Keras Model

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [6]:
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(4,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))
    
model.summary()
json_model = model.to_json()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                50        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


## Load Keras Model (SQLMagic method)

In [7]:
qmodel =  QuotedString(json_model)

In [8]:
%%sql 
DROP TABLE IF EXISTS model_arch_library;
SELECT madlib.load_keras_model('model_arch_library',            -- Output table
                                 {qmodel}::JSON,  -- Model architecture
                                 NULL,
                                 'Domino',                          -- Name of model
                                 'A simple model'                   -- Model description
);

 * postgresql://dominovaldano@localhost:6000/mnist
Done.
1 rows affected.


,load_keras_model
0,


In [9]:
%sql SELECT * FROM model_arch_library;

 * postgresql://dominovaldano@localhost:6000/mnist
1 rows affected.


,model_id,model_arch,model_weights,name,description,__internal_madlib_id__
0,1,"{'class_name': 'Sequential', 'config': {'name'...",None,Domino,A simple model,__madlib_temp_96012004_1601508886_68634802__


## Load Keras Model (psycopg2 method)

In [10]:
cur.execute("""
    SELECT madlib.load_keras_model('model_arch_library',            -- Output table
                                 %s::JSON,  -- Model architecture
                                 NULL,
                                 'Domino',                          -- Name of model
                                 'A simple model'                   -- Model description
    )
""",
    [json_model]
    )

In [11]:
%sql SELECT model_id, model_arch, name, description FROM model_arch_library

 * postgresql://dominovaldano@localhost:6000/mnist
2 rows affected.


,model_id,model_arch,name,description
0,2,"{'class_name': 'Sequential', 'config': {'name'...",Domino,A simple model
1,1,"{'class_name': 'Sequential', 'config': {'name'...",Domino,A simple model


## Run predict

In [ ]:
%%sql
SELECT madlib.madlib_keras_predict('places100_mult_model_10iters',
                                   'places100_test', 'id', 'x',
                                   'places100_predictions', '0.05',
                                   False, 5);

## Check/Fix Connection Status

In [12]:
from psycopg2 import extensions as ext

In [13]:
status_codes = { ext.STATUS_PREPARED : 'PREPARED', 
                 ext.STATUS_READY : 'READY',
                 ext.STATUS_IN_TRANSACTION : 'IN_TRANSACTION'
               }

In [21]:
print(status_codes[conn.status])

READY


In [22]:
cur.execute("SELECT 1")

In [23]:
cur.fetchall()

[(1,)]

In [24]:
conn.reset()
conn.autocommit = True  # Always reverts back to False after a reset

## Below is from Frank's Notebook

<a id="load_model_wts"></a>
## 3.  Load model weights

<a id="load_model_wts_madlib"></a>
### 3a. Load weights from previous MADlib run

Use UPDATE to load directly into the table.  For example, if 'model_weights' are the weights in the output table 'iris_model' from a previous run of 'madlib_keras_fit()' :

In [ ]:
%%sql
UPDATE model_arch_library
SET model_weights = iris_model.model_weights
FROM iris_model
WHERE model_arch_library.model_id = 2;

-- Check weights loaded OK
SELECT COUNT(*) FROM model_arch_library WHERE model_weights IS NOT NULL;

<a id="load_model_wts_keras1"></a>
### 3b. Load weights from Keras using a PL/Python function 
We need to flatten then serialize the weights to store as a PostgreSQL binary data type.  Byte format is more efficient on space and memory compared to a numeric array.  The model weights will be de-serialized when passed to Keras functions.

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION load_weights() RETURNS VOID AS
$$
from keras.layers import *
from keras import Sequential
import numpy as np
import plpy

# create model
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(4,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))

# get weights, flatten and serialize
weights = model.get_weights()
weights_flat = [w.flatten() for w in weights]
weights1d =  np.concatenate(weights_flat).ravel()
weights_bytea = weights1d.tostring()

# load query
load_query = plpy.prepare("""SELECT madlib.load_keras_model(
                        'model_arch_library',
                        $1, $2, $3, $4)
                    """, ['json','bytea', 'text', 'text'])
plpy.execute(load_query, [model.to_json(), weights_bytea, "Ella", "Model x"])
$$ language plpythonu;

-- Call load function
SELECT load_weights();

-- Check weights loaded OK
SELECT COUNT(*) FROM model_arch_library WHERE model_weights IS NOT NULL;

In [ ]:
%%sql
SELECT model_id, name from model_arch_library ORDER BY model_id;

<a id="load_model_wts_keras2"></a>
### 3c. Load weights from Keras using psycopg2
Psycopg is a PostgreSQL database adapter for the Python programming language.  As above we need to flatten then serialize the weights to store as a PostgreSQL binary data type.

Note that the psycopg2.Binary function will increase the size of the Python object for the weights, so if your model is large it might be better to use a PL/Python function as in 3b. above.

In [ ]:
import psycopg2 as p2
#conn = p2.connect('postgresql://gpadmin@35.239.240.26:5432/madlib')
conn = p2.connect('postgresql://fmcquillan@localhost:5432/madlib')
cur = conn.cursor()

from keras.layers import *
from keras import Sequential
import numpy as np

# create model
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(4,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))

# get weights, flatten and serialize
weights = model.get_weights()
weights_flat = [w.flatten() for w in weights]
weights1d =  np.concatenate(weights_flat).ravel()
weights_bytea = p2.Binary(weights1d.tostring())

query = "SELECT madlib.load_keras_model('model_arch_library', %s,%s,%s,%s)"
cur.execute(query,[model.to_json(), weights_bytea, "Grace", "Model y"])
conn.commit()

# check weights loaded OK
%sql SELECT COUNT(*) FROM model_arch_library WHERE model_weights IS NOT NULL;

In [ ]:
%%sql
SELECT model_id, name from model_arch_library ORDER BY model_id;